In [1]:
import csv
import random

input_file = 'Y_train.csv'
with open(input_file, 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

title = data[0]

rows = data[1:]

for i in range(24):
    extracted_rows = [rows[j] for j in range(len(rows)) if (j - i) % 24 == 0]
    
    output_file = f'./generated_data/output_{i+1}.csv'
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(title)
        writer.writerows(extracted_rows)

    with open(output_file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    
    title_row = data[0]
    rows_data = data[1:]
    
    for row in rows_data:
        original_value = float(row[0])
        for j in range(1, 2):
            random_factor = random.uniform(-0.15, 0.15)
            new_value = original_value * (1 + random_factor)
            row.append(f"{new_value:.2f}")
    
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(title_row + ['Change1'])
        writer.writerows(rows_data)


In [2]:
import csv
import numpy as np
import os

mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]

ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1] / sum(mpc_bus[j][1] for j in range(118))

parent_folder = './hour'
if not os.path.exists(parent_folder):
    os.makedirs(parent_folder)

for file_num in range(1, 25):
    input_file = f'./generated_data/output_{file_num}.csv'
    with open(input_file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)

    title = data[0]
    rows = data[1:]

    folder_path = f'{parent_folder}/{file_num}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    allocated_data = []
    for row in rows:
        total_value = float(row[0]) 
        allocated_row = [total_value * ratio_bus[i] for i in range(118)]
        allocated_data.append(allocated_row)

    output_file = f'{folder_path}/allocated_output_perfect.csv'
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Node_' + str(i+1) for i in range(118)])  
        writer.writerows(allocated_data)

    for col in range(1, 2):
        allocated_data = []
        for row in rows:
            total_value = float(row[col])  
            allocated_row = [total_value * ratio_bus[i] for i in range(118)]
            allocated_data.append(allocated_row)

        output_file = f'{folder_path}/allocated_output_change{col}.csv'
        with open(output_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Node_' + str(i+1) for i in range(118)])  
            writer.writerows(allocated_data)



In [3]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import os
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

def calculate_optimization_variables(P_l_predict, Y_real, topo, unit, reserve_up=150, reserve_down=150):
    P_l_predict = np.array(P_l_predict).T    
    Y_real = np.array(Y_real).T
    
    nodes = topo.shape[0]
    
    model_DA = gp.Model()
    
    P_c = model_DA.addVars(nodes, lb=0, name='P_c')       
    R_U = model_DA.addVars(nodes, lb=0, name='R_U')       
    R_D = model_DA.addVars(nodes, lb=0, name='R_D')       
    delta_DA = model_DA.addVars(nodes, lb=-180, ub=180, name='delta_DA') 
    
    obj_DA = gp.quicksum(
        R_U[i] * unit.iloc[i, 5] + 
        R_D[i] * unit.iloc[i, 6] + 
        P_c[i] * unit.iloc[i, 2] 
        for i in range(nodes) 
    )
    model_DA.setObjective(obj_DA, GRB.MINIMIZE)
    
    model_DA.addConstr(gp.quicksum(R_U[i] for i in range(nodes)) == reserve_up)
    model_DA.addConstr(gp.quicksum(R_D[i] for i in range(nodes)) == reserve_down)
        
    for i in range(nodes):
        model_DA.addConstr(R_U[i] <= unit.iloc[i, 3])  
        model_DA.addConstr(R_D[i] <= unit.iloc[i, 4])  
        
    for t in range(nodes):
        model_DA.addConstr(
            P_c[t] - P_l_predict[t] - 
            gp.quicksum(topo.iloc[t, j] * (delta_DA[t] - delta_DA[j]) for j in range(nodes)) == 0
        )
    model_DA.addConstr(delta_DA[0] == 0)  
        
    for i in range(nodes):
        model_DA.addConstr(P_c[i] <= unit.iloc[i, 1] - R_U[i])  
        model_DA.addConstr(P_c[i] >= R_D[i])                   
        
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i, j] < -0.01:
                model_DA.addConstr(topo.iloc[i, j] * (delta_DA[i] - delta_DA[j]) <= 175)
                model_DA.addConstr(topo.iloc[i, j] * (delta_DA[i] - delta_DA[j]) >= -175)
    
    model_DA.optimize()
    
    P_c_values = [P_c[i].X for i in range(nodes)]
    R_U_values = [R_U[i].X for i in range(nodes)]
    R_D_values = [R_D[i].X for i in range(nodes)]
    delta_DA_values = [delta_DA[i].X for i in range(nodes)]
    
    decision_variables = {
        'P_c': P_c_values,
        'R_U': R_U_values,
        'R_D': R_D_values,
        'delta_DA': delta_DA_values
    }
    
    model_DA.dispose()
    
    return decision_variables

topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) 
unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit')
hour_folder = './hour'

for hour in range(2, 25):
    hour_dir = os.path.join(hour_folder, str(hour))
    if not os.path.exists(hour_dir):
        continue
    
    perfect_file = os.path.join(hour_dir, 'allocated_output_perfect.csv')
    if not os.path.exists(perfect_file):
        continue
    
    with open(perfect_file, 'r') as f_perfect:
        reader_perfect = csv.reader(f_perfect)
        next(reader_perfect)  
        perfect_data = [list(map(float, row)) for row in reader_perfect]
    
    decision_variables_list = []
    with ThreadPoolExecutor() as executor:
        decision_variables_list = list(executor.map(lambda x: calculate_optimization_variables(x, x, topo, unit), perfect_data))
    
    decision_variables_file = os.path.join(hour_dir, 'decision_variables.csv')
    with open(decision_variables_file, 'w', newline='') as f_decision:
        writer = csv.writer(f_decision)

        writer.writerow(['P_c', 'R_U', 'R_D', 'delta_DA'])

        for dv in decision_variables_list:
            writer.writerow([dv['P_c'], dv['R_U'], dv['R_D'], dv['delta_DA']])
    

    print(f"Hour {hour}: Decision variables calculated and saved to {decision_variables_file}")

Set parameter Username
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-30
Set parameter Username
Set parameter Username
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-30
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-30
Academic license - for non-commercial use only - expires 2026-03-30
Academic license - for non-commercial use only - expires 2026-03-30
Academic license - for non-commercial use only - expires 2026-03-30
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))
CPU model: 13th Gen Intel(R) Core(TM) i5-13400F, instruction set [SSE2|AVX|AVX2]
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))
Gurobi Optimizer version 11.0.0 build v1

KeyboardInterrupt: 

In [4]:
import os
import pandas as pd

def split_and_save_data(X_train_file, Y_train_file, output_dir):
    
    X_train = pd.read_csv(X_train_file)
    Y_train = pd.read_csv(Y_train_file)

    
    if len(X_train) != len(Y_train):
        raise ValueError("error, check")

    
    os.makedirs(output_dir, exist_ok=True)

    
    for i in range(24):
        
        extracted_X = X_train[(X_train.index - i) % 24 == 0]
        extracted_Y = Y_train[(Y_train.index - i) % 24 == 0]

        
        if len(extracted_X) != len(extracted_Y):
            raise ValueError(f"{i}hour error")

        
        X_output_file = os.path.join(output_dir, f'X_train_hour_{i + 1}.csv')
        Y_output_file = os.path.join(output_dir, f'Y_train_hour_{i + 1}.csv')
        extracted_X.to_csv(X_output_file, index=False)
        extracted_Y.to_csv(Y_output_file, index=False)


X_train_file = './X_train.csv'  
Y_train_file = './Y_train.csv'  
output_dir = './hourly_data'    


split_and_save_data(X_train_file, Y_train_file, output_dir)

In [5]:
import pandas as pd
import os


def split_test_data_by_hour(X_test_file, Y_test_file, output_folder, start_hour=7):
    
    X_test = pd.read_csv(X_test_file)
    Y_test = pd.read_csv(Y_test_file)

    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    
    for hour in range(24):
        
        actual_hour = (start_hour - 1 + hour) % 24 + 1

        
        extracted_X = X_test.iloc[hour::24, :]
        extracted_Y = Y_test.iloc[hour::24, :]

        
        extracted_X.to_csv(f'{output_folder}/X_test_hour_{actual_hour}.csv', index=False)
        extracted_Y.to_csv(f'{output_folder}/Y_test_hour_{actual_hour}.csv', index=False)


def main():
    
    X_test_file = 'X_test.csv'
    Y_test_file = 'Y_test.csv'

    
    output_folder = './test_split'

    
    split_test_data_by_hour(X_test_file, Y_test_file, output_folder, start_hour=7)


main()

In [6]:
import os
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import joblib
import ast  
import numpy as np


hourly_data_dir = './hourly_data'  
decision_variables_dir = './hour'   
model_output_dir = './hourly_models_scaler'  
scaler_output_dir = './hourly_scalers'  


os.makedirs(model_output_dir, exist_ok=True)
os.makedirs(scaler_output_dir, exist_ok=True)


for hour in range(1, 25):
    
    X_train_file = os.path.join(hourly_data_dir, f'X_train_hour_{hour}.csv')
    if not os.path.exists(X_train_file):
        print(f"file {X_train_file} not exist, skip")
        continue
    X_train = pd.read_csv(X_train_file)
    
    
    decision_variables_file = os.path.join(decision_variables_dir, str(hour), 'decision_variables.csv')
    if not os.path.exists(decision_variables_file):
        print(f"file {decision_variables_file} not exist, skip")
        continue
    decision_variables = pd.read_csv(decision_variables_file)
    
    
    if len(X_train) != len(decision_variables):
        print(f"hour {hour} skip")
        continue
    
    
    try:
        decision_variables['P_c'] = decision_variables['P_c'].apply(ast.literal_eval)
        decision_variables['R_U'] = decision_variables['R_U'].apply(ast.literal_eval)
        decision_variables['R_D'] = decision_variables['R_D'].apply(ast.literal_eval)
    except (ValueError, SyntaxError) as e:
        print(f"hour {hour} wrong{e}")
        continue
    
    
    P_c = np.array(decision_variables['P_c'].tolist())
    R_U = np.array(decision_variables['R_U'].tolist())
    R_D = np.array(decision_variables['R_D'].tolist())
    
    
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    
    scaler_P_c = StandardScaler()
    P_c_scaled = scaler_P_c.fit_transform(P_c)
    
    scaler_R_U = StandardScaler()
    R_U_scaled = scaler_R_U.fit_transform(R_U)
    
    scaler_R_D = StandardScaler()
    R_D_scaled = scaler_R_D.fit_transform(R_D)
    
    
    print(f"training hour {hour}  P_c ...")
    model_P_c = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=10000, random_state=42, verbose=True)
    model_P_c.fit(X_train_scaled, P_c_scaled)
    
    print(f"traing hour {hour}  R_U ...")
    model_R_U = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=10000, random_state=42, verbose=True)
    model_R_U.fit(X_train_scaled, R_U_scaled)
    
    print(f"traing hour {hour}  R_D ...")
    model_R_D = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=10000, random_state=42, verbose=True)
    model_R_D.fit(X_train_scaled, R_D_scaled)
    
    
    model_P_c_file = os.path.join(model_output_dir, f'model_P_c_hour_{hour}.pkl')
    model_R_U_file = os.path.join(model_output_dir, f'model_R_U_hour_{hour}.pkl')
    model_R_D_file = os.path.join(model_output_dir, f'model_R_D_hour_{hour}.pkl')
    
    scaler_X_file = os.path.join(scaler_output_dir, f'scaler_X_hour_{hour}.pkl')
    scaler_P_c_file = os.path.join(scaler_output_dir, f'scaler_P_c_hour_{hour}.pkl')
    scaler_R_U_file = os.path.join(scaler_output_dir, f'scaler_R_U_hour_{hour}.pkl')
    scaler_R_D_file = os.path.join(scaler_output_dir, f'scaler_R_D_hour_{hour}.pkl')
    
    joblib.dump(model_P_c, model_P_c_file)
    joblib.dump(model_R_U, model_R_U_file)
    joblib.dump(model_R_D, model_R_D_file)
    
    joblib.dump(scaler_X, scaler_X_file)
    joblib.dump(scaler_P_c, scaler_P_c_file)
    joblib.dump(scaler_R_U, scaler_R_U_file)
    joblib.dump(scaler_R_D, scaler_R_D_file)
    

training hour 1  P_c ...
Iteration 1, loss = 0.10933198
Iteration 2, loss = 0.10322447
Iteration 3, loss = 0.09803545
Iteration 4, loss = 0.09360381
Iteration 5, loss = 0.08984536
Iteration 6, loss = 0.08663228
Iteration 7, loss = 0.08391077
Iteration 8, loss = 0.08162524
Iteration 9, loss = 0.07967028
Iteration 10, loss = 0.07797374
Iteration 11, loss = 0.07650276
Iteration 12, loss = 0.07521468
Iteration 13, loss = 0.07406924
Iteration 14, loss = 0.07302310
Iteration 15, loss = 0.07204761
Iteration 16, loss = 0.07111320
Iteration 17, loss = 0.07020593
Iteration 18, loss = 0.06933085
Iteration 19, loss = 0.06846747
Iteration 20, loss = 0.06760295
Iteration 21, loss = 0.06673791
Iteration 22, loss = 0.06586352
Iteration 23, loss = 0.06497491
Iteration 24, loss = 0.06408059
Iteration 25, loss = 0.06317339
Iteration 26, loss = 0.06225054
Iteration 27, loss = 0.06130824
Iteration 28, loss = 0.06034693
Iteration 29, loss = 0.05936254
Iteration 30, loss = 0.05835229
Iteration 31, loss = 0.0

In [7]:
import os
import pandas as pd
import joblib
import numpy as np


test_split_dir = './test_split'  
model_output_dir = './hourly_models_scaler'  
scaler_output_dir = './hourly_scalers'  
results_output_dir = './test_results_scaler'  


os.makedirs(results_output_dir, exist_ok=True)


for hour in range(1, 25):
    
    X_test_file = os.path.join(test_split_dir, f'X_test_hour_{hour}.csv')
    if not os.path.exists(X_test_file):
        print(f"file {X_test_file} not exist, skip")
        continue
    X_test = pd.read_csv(X_test_file)
    
    
    model_P_c_file = os.path.join(model_output_dir, f'model_P_c_hour_{hour}.pkl')
    model_R_U_file = os.path.join(model_output_dir, f'model_R_U_hour_{hour}.pkl')
    model_R_D_file = os.path.join(model_output_dir, f'model_R_D_hour_{hour}.pkl')
    
    if not os.path.exists(model_P_c_file) or not os.path.exists(model_R_U_file) or not os.path.exists(model_R_D_file):
        print(f"hour {hour} skip")
        continue
    
    model_P_c = joblib.load(model_P_c_file)
    model_R_U = joblib.load(model_R_U_file)
    model_R_D = joblib.load(model_R_D_file)
    
    
    scaler_X_file = os.path.join(scaler_output_dir, f'scaler_X_hour_{hour}.pkl')
    scaler_P_c_file = os.path.join(scaler_output_dir, f'scaler_P_c_hour_{hour}.pkl')
    scaler_R_U_file = os.path.join(scaler_output_dir, f'scaler_R_U_hour_{hour}.pkl')
    scaler_R_D_file = os.path.join(scaler_output_dir, f'scaler_R_D_hour_{hour}.pkl')
    
    if not os.path.exists(scaler_X_file) or not os.path.exists(scaler_P_c_file) or not os.path.exists(scaler_R_U_file) or not os.path.exists(scaler_R_D_file):
        print(f"hour {hour} skip")
        continue
    
    scaler_X = joblib.load(scaler_X_file)
    scaler_P_c = joblib.load(scaler_P_c_file)
    scaler_R_U = joblib.load(scaler_R_U_file)
    scaler_R_D = joblib.load(scaler_R_D_file)
    
    
    X_test_scaled = scaler_X.transform(X_test)
    
 
    P_c_pred_scaled = model_P_c.predict(X_test_scaled)
    R_U_pred_scaled = model_R_U.predict(X_test_scaled)
    R_D_pred_scaled = model_R_D.predict(X_test_scaled)
    
    
    P_c_pred = scaler_P_c.inverse_transform(P_c_pred_scaled)
    R_U_pred = scaler_R_U.inverse_transform(R_U_pred_scaled)
    R_D_pred = scaler_R_D.inverse_transform(R_D_pred_scaled)
    
   
    results = pd.DataFrame({
        'P_c_pred': list(P_c_pred),
        'R_U_pred': list(R_U_pred),
        'R_D_pred': list(R_D_pred)
    })
    
  
    results_file = os.path.join(results_output_dir, f'test_results_hour_{hour}.csv')
    results.to_csv(results_file, index=False)


In [8]:
import pandas as pd
import os
import numpy as np


test_results_dir = './test_results_scaler'  
nodes_data_file = './118nodes_system.xlsx'  


unit_data = pd.read_excel(nodes_data_file, sheet_name='unit')


R_U_max = unit_data['Upward reserve capacity offer [MW]'].values.astype(float)  
R_D_max = unit_data['Downward reserve capacity offer [MW]'].values.astype(float)  


for hour in range(1, 25):
    
    results_file = os.path.join(test_results_dir, f'test_results_hour_{hour}.csv')
    if not os.path.exists(results_file):
        print(f"file {results_file} not exist")
        continue
    results = pd.read_csv(results_file)
    
    
    results['P_c_pred'] = results['P_c_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    results['R_U_pred'] = results['R_U_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    results['R_D_pred'] = results['R_D_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    
    
    results['R_U_pred'] = results['R_U_pred'].apply(lambda x: np.minimum(x, R_U_max))  
    results['R_U_pred'] = results['R_U_pred'].apply(lambda x: np.maximum(x, 0))  
    results['R_D_pred'] = results['R_D_pred'].apply(lambda x: np.minimum(x, R_D_max))  
    results['R_D_pred'] = results['R_D_pred'].apply(lambda x: np.maximum(x, 0))  

    
    results['R_U_pred'] = results['R_U_pred'].apply(lambda x: x * 150 / np.sum(x) if np.sum(x) != 0 else x)
    results['R_D_pred'] = results['R_D_pred'].apply(lambda x: x * 150 / np.sum(x) if np.sum(x) != 0 else x)
    
    
    results.to_csv(os.path.join(test_results_dir, f'test_results_hour_{hour}_processed.csv'), index=False)



In [9]:
import pandas as pd
import os
import numpy as np


test_results_dir = './test_results_scaler'
nodes_data_file = './118nodes_system.xlsx' 


unit_data = pd.read_excel(nodes_data_file, sheet_name='unit')


P_max = unit_data['Installed capacity [MW]'].values.astype(float)
R_U_max = unit_data['Upward reserve capacity offer [MW]'].values.astype(float) 
R_D_max = unit_data['Downward reserve capacity offer [MW]'].values.astype(float) 


for hour in range(1, 25):
    
    results_file = os.path.join(test_results_dir, f'test_results_hour_{hour}_processed.csv')
    if not os.path.exists(results_file):
        print(f"file {results_file} not exist")
        continue
    results = pd.read_csv(results_file)

    
    results['P_c_pred'] = results['P_c_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    results['R_U_pred'] = results['R_U_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    results['R_D_pred'] = results['R_D_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

    
    for i in range(len(results)):
        P_c_pred = results['P_c_pred'].iloc[i].copy()  
        R_U_pred = results['R_U_pred'].iloc[i]  
        R_D_pred = results['R_D_pred'].iloc[i] 

       
        for j in range(118): 
            P_c_pred[j] = min(P_c_pred[j], P_max[j] - R_U_pred[j])  
            P_c_pred[j] = max(P_c_pred[j], 0) 

        
        results.at[i, 'P_c_pred'] = P_c_pred

    
    results.to_csv(os.path.join(test_results_dir, f'test_results_hour_{hour}_final.csv'), index=False)



In [10]:
import pandas as pd
import os
import numpy as np

def combine_and_sort_predictions(test_results_dir, test_splits_dir):

   
    all_p_c = []
    all_r_u = []
    all_r_d = []
    all_trends = []

    
    for hour in range(1, 25):
        
        results_file = os.path.join(test_results_dir, f'test_results_hour_{hour}_final.csv')
        if not os.path.exists(results_file):
            print(f"file {results_file} not exist")
            continue
        results_df = pd.read_csv(results_file)
        
        
        X_test_file = os.path.join(test_splits_dir, f'X_test_hour_{hour}.csv')
        if not os.path.exists(X_test_file):
            print(f"file {results_file} not exist")
            continue
        X_test = pd.read_csv(X_test_file)
        
        
        results_df['P_c_pred'] = results_df['P_c_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
        results_df['R_U_pred'] = results_df['R_U_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
        results_df['R_D_pred'] = results_df['R_D_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
        
       
        p_c = np.vstack(results_df['P_c_pred'].values)
        r_u = np.vstack(results_df['R_U_pred'].values)
        r_d = np.vstack(results_df['R_D_pred'].values)
        
        
        trend = X_test['trend'].values
        
       
        all_p_c.append(p_c)
        all_r_u.append(r_u)
        all_r_d.append(r_d)
        all_trends.append(trend)

    
    all_p_c = np.vstack(all_p_c)
    all_r_u = np.vstack(all_r_u)
    all_r_d = np.vstack(all_r_d)
    all_trends = np.concatenate(all_trends)

    
    sorted_indices = np.argsort(all_trends)
    sorted_p_c = all_p_c[sorted_indices]
    sorted_r_u = all_r_u[sorted_indices]
    sorted_r_d = all_r_d[sorted_indices]
    sorted_trends = all_trends[sorted_indices]

    
    test_results = pd.DataFrame({
        'P_c_pred': [str(row) for row in sorted_p_c],
        'R_U_pred': [str(row) for row in sorted_r_u],
        'R_D_pred': [str(row) for row in sorted_r_d],
        'trend': sorted_trends
    })

    return test_results


test_results_dir = './test_results_scaler'  
test_splits_dir = './test_split'  


test_results = combine_and_sort_predictions(test_results_dir, test_splits_dir)


test_results.to_csv(os.path.join(test_results_dir, 'test_results_combined.csv'), index=False)


In [11]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

def load_da_results(filename='./test_results_scaler/test_results_combined.csv'):

    da_results = pd.read_csv(filename)
    
    da_results['P_c_pred'] = da_results['P_c_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    da_results['R_U_pred'] = da_results['R_U_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    da_results['R_D_pred'] = da_results['R_D_pred'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
    return da_results

def calculate_obj_DA(da_results, unit, time_steps, nodes):
    
    obj_DA = 0
    for k in range(time_steps):
        for i in range(nodes):
            P_c = da_results['P_c_pred'].iloc[k][i]
            R_U = da_results['R_U_pred'].iloc[k][i]
            R_D = da_results['R_D_pred'].iloc[k][i]
            obj_DA += P_c * unit.iloc[i, 2] + R_U * unit.iloc[i, 5] + R_D * unit.iloc[i, 6]

    return obj_DA

def save_optimization_results(model_RT, time_steps, nodes, filename='optimization_results_scaler.csv'):
  
    rt_results = []
    for k in range(time_steps):
        for i in range(nodes):
            rt_results.append([
                'RT', k, i,
                model_RT.getVarByName(f'r_U[{i},{k}]').X,
                model_RT.getVarByName(f'r_D[{i},{k}]').X,
                model_RT.getVarByName(f'P_lsh[{i},{k}]').X,
                model_RT.getVarByName(f'delta_RT[{i},{k}]').X,
                model_RT.getVarByName(f's[{i},{k}]').X
            ])
    rt_columns = ['Stage', 'TimeStep', 'Node', 'r_U', 'r_D', 'P_lsh', 'delta_RT', 's']
    df_rt = pd.DataFrame(rt_results, columns=rt_columns)
    df_rt.to_csv(filename, index=False)

def calculate_optimization_cost_two_stage(Y_real, topo, unit, ratio_bus, reserve_up=150, reserve_down=150, c=9000, c_s=1750):
    

    da_results = load_da_results()
    nodes = topo.shape[0]
    time_steps = Y_real.shape[0]


    obj_DA = calculate_obj_DA(da_results, unit, time_steps, nodes)
    print(f"day_ahead cost: {obj_DA}")

    Y_real_allocated = allocate_data_to_nodes(Y_real, ratio_bus).T

    model_RT = gp.Model()
    r_U = model_RT.addVars(nodes, time_steps, lb=0, name='r_U')
    r_D = model_RT.addVars(nodes, time_steps, lb=0, name='r_D')
    P_lsh = model_RT.addVars(nodes, time_steps, lb=0, name='P_lsh')  
    delta_RT = model_RT.addVars(nodes, time_steps, lb=-180, ub=180, name='delta_RT')
    s = model_RT.addVars(nodes, time_steps, lb=0, name='s')  

    
    obj_RT = gp.quicksum(
        (r_U[i, j] - r_D[i, j]) * unit.iloc[i, 2]
        for i in range(nodes) for j in range(time_steps)
    )
    obj_RT += gp.quicksum(P_lsh[i, j] * c for i in range(nodes) for j in range(time_steps))
    obj_RT += gp.quicksum(s[i, j] * c_s for i in range(nodes) for j in range(time_steps))
    model_RT.setObjective(obj_RT, GRB.MINIMIZE)

  
    for k in range(time_steps):
      
        for t in range(nodes):
            model_RT.addConstr(
                r_U[t, k] - r_D[t, k] - s[t, k] - Y_real_allocated[t, k] + P_lsh[t, k] -
                gp.quicksum(
                    topo.iloc[t, j] * (delta_RT[t, k] - delta_RT[j, k])
                    for j in range(nodes))
                + da_results['P_c_pred'].iloc[k][t] 
                == 0
            )
       
        for i in range(nodes):
            model_RT.addConstr(r_U[i, k] <= da_results['R_U_pred'].iloc[k][i])  
            model_RT.addConstr(r_D[i, k] <= da_results['R_D_pred'].iloc[k][i])  
        model_RT.addConstr(delta_RT[0, k] == 0) 
        
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i, j] < -0.01:
                    model_RT.addConstr(topo.iloc[i, j] * (delta_RT[i, k] - delta_RT[j, k]) <= 175)
                    model_RT.addConstr(topo.iloc[i, j] * (delta_RT[i, k] - delta_RT[j, k]) >= -175)

    model_RT.optimize()

    total_cost = obj_DA + model_RT.ObjVal
    print(f"total cost: {total_cost}")

    save_optimization_results(model_RT, time_steps, nodes, 'optimization_results_scaler.csv')
    return total_cost


def allocate_data_to_nodes(data, ratio_bus):
    
    allocated_data = np.zeros((data.shape[0], 118))
    for i in range(data.shape[0]):
        allocated_data[i, :] = float(data[i]) * ratio_bus
    return allocated_data


if __name__ == "__main__":
    Y_real = pd.read_csv('Y_test.csv').values.flatten()

    topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None)
    unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit')

    mpc_bus = [
        [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
        [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
        [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
        [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
        [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
        [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
        [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
        [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
        [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
        [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
        [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
        [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
    ]
    ratio_bus = np.zeros(118)
    for i in range(118):
        ratio_bus[i] = mpc_bus[i][1] / sum(mpc_bus[j][1] for j in range(118))

    total_cost = calculate_optimization_cost_two_stage(Y_real, topo, unit, ratio_bus)

day_ahead cost: 45777433.59835531
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13400F, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 584766 rows, 322140 columns and 1428882 nonzeros
Model fingerprint: 0xa9ecfe5a
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [4e+01, 9e+03]
  Bounds range     [2e+02, 2e+02]
  RHS range        [3e-06, 3e+02]
Presolve removed 426377 rows and 113120 columns
Presolve time: 1.55s
Presolved: 158389 rows, 305662 columns, 690783 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.08s

Barrier statistics:
 AA' NZ     : 1.266e+06
 Factor NZ  : 2.715e+06 (roughly 200 MB of memory)
 Factor Ops : 4.997e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Re